In [1]:
import xarray as xr
import numpy as np
import cftime

In [2]:
start = '2010-01-01'
end = '2020-12-31'

# Create noleap datarray
times = xr.cftime_range(start=start, end=end, freq='D', calendar='noleap')
data = xr.DataArray(np.arange(times.size * 4).reshape(times.size, 2, 2) ,dims=['time', 'lat', 'lon'], coords={'time': times})
data

<xarray.DataArray (time: 4015, lat: 2, lon: 2)>
array([[[    0,     1],
        [    2,     3]],

       [[    4,     5],
        [    6,     7]],

       [[    8,     9],
        [   10,    11]],

       ...,

       [[16048, 16049],
        [16050, 16051]],

       [[16052, 16053],
        [16054, 16055]],

       [[16056, 16057],
        [16058, 16059]]])
Coordinates:
  * time     (time) object 2010-01-01 00:00:00 ... 2020-12-31 00:00:00
Dimensions without coordinates: lat, lon

In [3]:
times_2 = xr.DataArray(xr.cftime_range(start=start, end=end, freq='D'),dims=['time'])
times_2

<xarray.DataArray (time: 4018)>
array([cftime.DatetimeGregorian(2010, 1, 1, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2010, 1, 2, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2010, 1, 3, 0, 0, 0, 0), ...,
       cftime.DatetimeGregorian(2020, 12, 29, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2020, 12, 30, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2020, 12, 31, 0, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 2010-01-01 00:00:00 ... 2020-12-31 00:00:00

In [4]:
cond = (times_2.time.dt.month == 2) & (times_2.time.dt.day == 29)

In [5]:
leap_days = times_2.where(cond, drop=True)
leap_days

<xarray.DataArray (time: 3)>
array([cftime.DatetimeGregorian(2012, 2, 29, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2016, 2, 29, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2020, 2, 29, 0, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 2012-02-29 00:00:00 ... 2020-02-29 00:00:00

In [6]:
def convert(time_obj):
    converted = cftime.DatetimeGregorian(year=time_obj.year, month=time_obj.month, day=time_obj.day)
    return converted


times_greg = [convert(time_obj) for time_obj in data.time.data]

In [7]:
data["time"] = times_greg

In [8]:
data

<xarray.DataArray (time: 4015, lat: 2, lon: 2)>
array([[[    0,     1],
        [    2,     3]],

       [[    4,     5],
        [    6,     7]],

       [[    8,     9],
        [   10,    11]],

       ...,

       [[16048, 16049],
        [16050, 16051]],

       [[16052, 16053],
        [16054, 16055]],

       [[16056, 16057],
        [16058, 16059]]])
Coordinates:
  * time     (time) object 2010-01-01 00:00:00 ... 2020-12-31 00:00:00
Dimensions without coordinates: lat, lon

In [9]:
one_time_step = data.isel(time=slice(0, 1))
one_time_step

<xarray.DataArray (time: 1, lat: 2, lon: 2)>
array([[[0, 1],
        [2, 3]]])
Coordinates:
  * time     (time) object 2010-01-01 00:00:00
Dimensions without coordinates: lat, lon

In [10]:
x = []
for leap_day in leap_days:
    d = xr.full_like(one_time_step,fill_value=np.nan)
    d = d.assign_coords(time=[leap_day.data])
    x.append(d)

In [11]:
x[0]

<xarray.DataArray (time: 1, lat: 2, lon: 2)>
array([[[-9223372036854775808, -9223372036854775808],
        [-9223372036854775808, -9223372036854775808]]])
Coordinates:
  * time     (time) object 2012-02-29 00:00:00
Dimensions without coordinates: lat, lon

In [12]:
x[1]

<xarray.DataArray (time: 1, lat: 2, lon: 2)>
array([[[-9223372036854775808, -9223372036854775808],
        [-9223372036854775808, -9223372036854775808]]])
Coordinates:
  * time     (time) object 2016-02-29 00:00:00
Dimensions without coordinates: lat, lon

In [13]:
data

<xarray.DataArray (time: 4015, lat: 2, lon: 2)>
array([[[    0,     1],
        [    2,     3]],

       [[    4,     5],
        [    6,     7]],

       [[    8,     9],
        [   10,    11]],

       ...,

       [[16048, 16049],
        [16050, 16051]],

       [[16052, 16053],
        [16054, 16055]],

       [[16056, 16057],
        [16058, 16059]]])
Coordinates:
  * time     (time) object 2010-01-01 00:00:00 ... 2020-12-31 00:00:00
Dimensions without coordinates: lat, lon

In [18]:
x.append(data)

ds=xr.concat(x, dim='time').sortby('time')
ds